In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc


##### Merge: periodos + clientes + productos

In [6]:

sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")
del sellin, productos, stocks

df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

periodos = pd.date_range(start=df['periodo_dt'].min(), end=df['periodo_dt'].max(), freq="MS")
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()

idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
completo["periodo"] = completo["periodo"].dt.strftime("%Y%m").astype(int)

del periodos, productos, clientes
gc.collect()
completo.head()

Ventas-Productos-Stocks: 2945818 filas y 13 columnas


,product_id,customer_id,periodo
0,20524,10234,201701
1,20524,10234,201702
2,20524,10234,201703
3,20524,10234,201704
4,20524,10234,201705


##### Cruzamos con productos

In [9]:
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
completo = completo.merge(productos, how='left', on="product_id")
del productos
gc.collect()

72

##### Cruzamos con stock

In [10]:
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
completo = completo.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks
gc.collect()
completo.head()

,product_id,customer_id,periodo,cat1,cat2,cat3,brand,sku_size,stock_final
0,20524,10234,201701,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,20524,10234,201702,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,20524,10234,201703,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,20524,10234,201704,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,20524,10234,201705,HC,VAJILLA,Cristalino,Importado,500.0,NaN


##### Cruzamos con ventas

In [11]:
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
# Agrupar ventas por periodo, cliente y producto
dt = sellin.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
df_completo = completo.merge(dt, how='left', on=['periodo', 'customer_id','product_id'])
df_completo['tn'] = df_completo['tn'].fillna(0)
del sellin, dt, completo
gc.collect()

40

##### Target

In [13]:
# Asegurarte de tener 'periodo_dt' (datetime) en completo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear DataFrame auxiliar con tn como target y fecha adelantada
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras['periodo_target_dt'] = ventas_futuras['periodo_dt'] - pd.DateOffset(months=2)
ventas_futuras = ventas_futuras.rename(columns={'tn': 'target'})

# Merge con completo usando periodo adelantado
df_completo = df_completo.merge(
    ventas_futuras[['periodo_target_dt', 'customer_id', 'product_id', 'target']],
    how='left',
    left_on=['periodo_dt', 'customer_id', 'product_id'],
    right_on=['periodo_target_dt', 'customer_id', 'product_id']
)

# Eliminar columna auxiliar
df_completo = df_completo.drop(columns=['periodo_target_dt'])

print(f"✅ Target generado. Filas con target no nulo: {df_completo['target'].notna().sum()}")

✅ Target generado. Filas con target no nulo: 25027434


##### Verifico las NaN en el target: Existen porque hay clientes que solo compraron 2 veces.

In [ ]:
nan_count = df_completo['target'].isna().sum()
print(f"🔍 Total de NaN en target: {nan_count}")

##### Generación de IDs

In [14]:
df_completo = df_completo.sort_values(['periodo', 'customer_id', 'product_id'])
df_completo['id'] = df_completo.groupby(['customer_id', 'product_id']).cumcount() + 1